- problem definition: 
Task 1: Minimise the overall delivery time and cost of delivery.
Task 2: Maximise cost-efficiency (total gas delivered/overall cost of delivery)
# In both cases: 
The task is to efficiently distribute fuel across a number of customer nodes. It is in mind to use each depot to their fullest potential through means such as clustering of nodes. The problem is at first glance, a combinatorial problem as it introduces the need for optimising many parameters to find the most optimal task route. 

- Constraints:
Task 1: Deliver to customers with less gas than 50% of their tank's capacity.
Task 2: Deliver to customers with less gas than 80% of their tank's capacity and fill up to 80% of their tank's capacity.

- optimization algorithm (+ others): 
Task 1: Greedy search algorithm, Breadth-First-Search algorithm
Task 2: Continuous Genetic Algorithm, Breadth-First-Search algorithm

- results:
outputted to task1_solution.json file 

- methodology:

1. Converting the .csv files into pandas dataframe (using pandas library)

Firstly I converted the data stored in .csv files into a pandas dataframe. There are many advantages to this (1), as well as some disadvantages (2): 
+ Python supported operations 
+ NumPy supported operations 
+ Extensive set of feature techniques 
+ Readable syntax 
+ Cleaner representation of data structures 
+ Conversion of .csv to a tabular format
- Documenation (less coverage on the harder functions of pandas library)

In [ ]:
# EXAMPLE CODE FOR .CSV FILE CONVERSION INTO PANDAS 

# relevant imports 
import pandas as pd 

# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# visual representation of location_df in tabular format
location_df.head()

2. Implementing the use of numPy library to process data series from a pandas dataframe

NumPy library deemed important for processing and operating on large data structures stored in the relative dataframes. The advantages and disadvantages of using NumPy library include (3): 

+ As opposed to the tabular data representation in Pandas used mainly for analysing data, NumPy offers multi-dimensional arrays and matrix datastructures for performing mathematical operations.
- Some numpy values can cause ambiguity issues in pythons interprerator and additional processing steps may be required.


In [ ]:
# CODE EXAMPLE FOR USING NUMPY FOR EXTRACTING PANDAS ROWS WHERE CONDITION IS MET

# relevant imports 
import numpy as np

# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0] 

3. Graph visualisation and euclidean distance

The networkx library was implemented to visualise the network of customer and depot nodes stored in the pandas dataframe.
Why networkx? 
+ supports mapping of nodes, edges with arbritrary data, and analysis of network structure (6).

The scipy library was used to support the calculation of euclidean distance betwene two nodes. This library supports matrix computation of vector observations that are stored in an array (7). This made the the extraction of spatial distance between any two given nodes a consistent process throughout both tasks 1 and 2. 

In [ ]:
# EXAMPLE CODE FOR MATRIX CALCULATION OF SPATIAL DISTANCE BETWEEN ALL NODES STORED IN THE ARRAY

# relevant imports 
from scipy.spatial.distance import pdist, squareform

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# displaying first 5 edges in the edges list
print(f'#EDGES# (nodeA, nodeB, distance): {edges[:5]}')

In [ ]:
# EXAMPLE CODE FOR VISUALISATION OF THE NETWORK

# relevant imports 
import networkx as nx 
import matplotlib.pyplot as plt

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

4. Clustering (based on the notion of nearest-neighbours)

Objective: finding the partition of data set, assigning each data (customer nodes) in the sample to four distinct points (depot nodes). 

A self-implemented clustering program was used for allocating customers to relative depots. This program takes into account the spatial distance between each customer node to all depot nodes, assigning them to the depot they are nearest to. In essence, the algorithm is 'based' on the notion of 'nearest-neighbour' because it is likely that closer nodes are likely to have less overall traversal time. However, this notion can be countered when taking road links into account, therefore, an algorithm that examines the complete breadth of a path (hence, why we have used breadth-first-search as discussed later on). 

In [ ]:
# EXAMPLE CODE FOR CLUSTERING OF NODES

# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

5. Breadth-First-Search (BFS) Algorithm ### PATH FINDING ###

A self-implemented pathfinding program based on the notion of breadth-first-search was used for retrieiving the shortest order of traversals between two nodes. The main idea is the use queue and visit every adjacent node to the most recent node visited. When the destination node has been reached, we can backtrace through all the relative traversals made between the starting node and the final node to get the shortest route between the nodes in the graph. In this context, since route id's are independent of node id's; 'edge memory' list was additionaly introduced to store the set of explored edges (used in backtracing as shown in the example code below).

+ nb: the distance between two nodes were featured as the edge weights.  

In [ ]:
# EXAMPLE CODE FOR BREADTH-FIRST-SEARCH ALGORITHM FOR PATHFINDING SHORTEST ROUTE BETWEEN TWO NODES

# Routing from A to B using recursive Breadth-First-Search based algorithm to find shortest route, with back tracing
def routing(currState, toState, edges):
    # intialise visitedState list for tracking node traversal
    visitedState = [currState]
    # initialise visitedEdge list for tracking edge traversal
    visitedEdge = []
    # intialise edgeMemory list for storing the explored edges
    edgeMemory = []
    # intialise queue list for choosing the central node for next traversal
    queue = [currState]

    # loop while toState is not found
    while currState != toState:
        # remove and store the last element of the queue list as q
        q = queue.pop(0)
        
        # intialise currEdges dict which holds the next set of edges for traversals
        currEdges = {}
        # get the nodes at each edge, where either nodes are equivalent to q
        for edge in list(np.where(links_df[['id1', 'id2']]==q)[0]):
            # update the dict with the relative edge key and the node pairs
            currEdges.update({edge:[edges[edge][0], edges[edge][1]]})
        
        # store the explored edges in edgeMemory list
        edgeMemory.append(currEdges)

        # loop through each edge in currEdges
        for edge in currEdges:
            # check if the edge has been visited 
            if edge not in visitedEdge:
                # if not visited then add the edge to visitedEdge
                visitedEdge.append(edge)
                # check the node index in the edge that has not been visited 
                if currEdges[edge][0] not in visitedState and currEdges[edge][1] in visitedState:
                    # set currState to the unvisited node 
                    currState = currEdges[edge][0]
                    # mark the node in currState as visited 
                    visitedState.append(currState)
                    # add new currState to queue 
                    queue.append(currState)
                    # check if toState reached
                    if currState == toState:
                        # set currState to toState
                        currState = toState
                        # end loop
                        break 
                # similar to above but in the context of different index position of the node that has not been visited
                if currEdges[edge][1] not in visitedState and edges[edge][0] in visitedState:
                    currState = currEdges[edge][1]
                    visitedState.append(currState)
                    queue.append(currState)
                    if currState == toState:
                        currState = toState
                        break
    
    # set startState as the first node in visitedState list
    startState = visitedState[0]
    # set lastQ as the toState for tracking q node from end of order
    lastQ = [toState]
    # intialise backtrace list for backtracing the edges from edgeMemory
    backtrace = []
    # initialise nodetrace list for backtracing the nodes from edgeMemory
    nodetrace = []

    # intialise edgeMemoryReversed for reordering edgeMemory 
    edgeMemoryReversed = []
    # loop through each index between range 0 and length of edgeMemory
    for i in range (0, len(edgeMemory)):
        # set endElement to the last element in edgeMemory
        endElement = edgeMemory.pop(-1)
        # add the endElement to edgeMemoryReversed
        edgeMemoryReversed.append(endElement)
    
    # while last element in lastQ is not equivalent to the startState
    while lastQ[-1] != startState:
        # loop through each edge options in edgeMemoryReversed
        for edgeOpt in edgeMemoryReversed:
            # loop through each edge from as keys of the edge options
            for edge in edgeOpt.keys():

                # check if last element of lastQ is in the set of edge options given the edge
                if lastQ[-1] in edgeOpt[edge]:
                    # add the edge to backtrace 
                    backtrace.append(edge)
                    # check index of node which matches the lastQ element 
                    if lastQ[-1] == edgeOpt[edge][0] and lastQ[-1] != edgeOpt[edge][1]:
                        # update lastQ as the the node which does not match the lastQ element
                        lastQ.append(edgeOpt[edge][1])
                        # add the node to nodetrace
                        nodetrace.append(edgeOpt[edge][1])
                        # return to while iterate
                        break
                        # similar to above but in the context of different index postion of the matching node with lastQ element
                    if lastQ[-1] != edgeOpt[edge][0] and lastQ[-1] == edgeOpt[edge][1]:
                        lastQ.append(edgeOpt[edge][0])
                        nodetrace.append(edgeOpt[edge][0])
                        break
    
    # re-ordering edges from start to end
    edgeTraversed = []
    for i in range(0, len(backtrace)):
        endElement = backtrace.pop(-1)
        edgeTraversed.append(endElement)

    # re-ordering nodes from start to end 
    nodeOrder = []
    for i in range(0, len(nodetrace)):
        endElement = nodetrace.pop(-1)
        nodeOrder.append(endElement)
    
    # adding route weight (distance between nodes) to each traversal made
    routeWeight = []
    for edge in edgeTraversed:
        routeWeight.append(edges[edge][2])

    # return the the order in which nodes were visited and the order in which edges were traversed
    return nodeOrder, edgeTraversed, routeWeight

# dummy print: for example purpose
print(f'nodes order: {routing(124, 10, edges)[0]}')
print(f'edges traversed: {routing(124, 10, edges)[1]}')
print(f'route weight: {routing(124, 10, edges)[2]}')

6. task 1
# greedy search

The greedy algorithm is a search strategy which makes the most optimal decision at each stage [10]. Problems related with small fraction of states, in which sparse-space need to be covered can be addressed by the greedy search strategy [11]. In this context, given a starting node, the next node is chosen via the relative spatial distance between each node. Greedy search strategy has been used as a simple means for minimising the distance travelled or dispatch time, especially since the problem dimension is published on a 2-dimensional (2D) plane.
# This concept has been applied in selecting the order in which each allocated customers per depot (nodes per cluster point) are visited.
For instance, the most susceptible next node being the closest node to the current node of consideration. This helps minimise distance. However, again since the overall path distance including traversals made from one point to next is not considered, it has blindspots for the distances between those paths, although it is included in the final result. 

# pseudo-code-architecture for the greedy search function: 

- parameters: current node, allocated nodes, spatial distance, destination node

- function structure: nearest_node(start node, goal node, edge weights) # outputs a destination node based on spatial distance

- formulation: destination node = nearest_node(current node, allocated nodes, relative spatial distances)
                                = 'relative spatial distance' between 'current node' and 'each node in the allocated nodes'
                                = 'the node pair with the shortest spatial distance' (where one of the node is the current node)

In [ ]:
# EXAMPLE CODE FOR GREEDY SEARCH FUNCTIONS

# function: finding next nearest customer node
def nearest_customer(currentState, customerList):
    #initialise dist dict
    dist = {}
    
    # loop through customerList
    for i in customerList:
        # check for all where customer != currentState
        if i != currentState:
            # update dist with available customer index and their relative weights
            dist.update({i:euclidean[i,currentState]})

    # initialise temp list
    temp = []

    # loop through keys of dist 
    for i in dist.keys():
        # add weights to temp
        temp.append(dist[i])
    
    # get lowest weight
    _shortestDist = min(temp)
    
    # find relative index of lowest weight
    nearestCustomerIndex = np.where(euclidean[currentState]==_shortestDist)
    
    # return next index with relative weight
    return int(nearestCustomerIndex[0]), _shortestDist

# dummy prints: for example purposes
print(f'nearest customer to node 124: {nearest_customer(124, cluster[124])[0]}')
print(f'distance: {nearest_customer(124, cluster[124])[1]}')

6. task 2
# genetic algorithm

Genetic algorithm searches for the fittest population in a given set of populations and introduces the notion of reproduction in aims of improving the solution. In doing so, the algroithm utilises the notion of natural evolution, which is why they are considered as a branch of evoltuionary algorithms (set of nature based algorithms for problem solving) [12]. Due to the context of the problem, genetic algorithm is an ideal approach for solving np-complete (combinatorial) problems in order to either maximise gain or minimise loss. 

- Complexity induces time consumption. In this context, using BFS algorithm for evaluating fitness in conjunction with the genetic algorithms fitness function increased the time taken to evaluate the fitness per genome (this will be further discussed in the results).

# pseudo-code-architecture for genetic algorithm functions: (defines problem formulation using genetic algorithm structure)

Although the code has been documented with inline comments, some general objectives of each function of the algorithm include:
- functions: generateGenome(), generatePopulation(), calculateFitness(), selectParents(), crossover(), mutation(), main()
NB: the functions in the actual code and this pseudo-code do not share the exact same names. 

- objectives: 

+ generateGenome() : The genome in this case is a solution. A solution in this context is the order of customer nodes per depot. The order is significant because it defines the order in which the lorry visits each location (similar to the problem of travelling salesman person). Therefore, it is possible to think of a solution as the order of route assigned per depot. The depots and node allocation in this instance be thought of as genes and chromosomes in the genome respectively. 

+ generatePopulation() : The population is represented by many number of genomes per set. This is equivalent to having many different number of solutions stored within a list. Some genomes in the populations are retained in future iterations, whereas some are replaced by newer genomes; allowing for the breadth of all possible routes to be explored. 

+ calculateFitness() : The fitness of a genome is the total weight of chromosomes per segment. In problem context, the fitness of the solution is the total distance of each route per depot, summed up to produce an overall distance for the solution. Any two solutions where routes are ordered differently may or may not produce the same fitness score. 

+ selectParents() : the selection function chooses two genomes (two solutions) as parents for the mating operation. The selection process in this case is based on the notion of roulette-wheel selection technique. This approach is constructed from the relative fitness of each individual genomes [13]. For instance, each genome is given a selection probability based on their fitness. By avoiding elitism (selecting only the best methods), I have attempted to split this algorithm from the greedy search which only picks the optimal solution at each stage. This is to avoids the risk of local minima but not entirely [14].

+ crossover() : the crossover function partitions the genomes are the same segment point. The opposite segments from each genomes are combined to create a new complete solution. In this case the algorithm implements an 'n point' crossover technique where a segment (depot) in the solution is randomly chosen as the cutting point. The solution is sliced and each partition of the solution is combined to the opposite part from the other solution. This is also known as the mating process.

+ mutation() : the mutaton function aims to alter the outcome of each solution by altering the solution itself. The mutation is only applied to the offsprings. It consists of a mutation rate which randomly decides whether the mutation should occur or not. In this context, 'n point' mutation technique has been implemented to randomly choose a depot in a solution. The order of route for that depot is rearranged in a random order in aims of producing a new solution. Since, crossovers would likely reproduce solutions that are similar, it is a risk for pre-mature convergence. Whereas, introducing this mutation function gives an opportunity for diversity in the offsprings, countering the risk of converging on a local minima. 

+ main() : this is a helper function used to run compute the functions in an asynchronous order per iteration. The code has been created with reusability in mind, and such the function can take in a number of variables such as size, iteration, and genome structure. The iteration refers to the termination criteria as well, in which the function loops until the maximum iteration has been met but not exceeded. 

+ other()'(s) : there are some additional functions that were made to support additional steps for this problem context; such as picking the best solutions per iteration, retaining the best solutions for the next generation, and replacing poorer solutions as well. These are documented inline with the code. 

In [ ]:
# dummyCluster is only generated here for example purpose
dummyCluster = {124: [10, 15, 37, 43, 71, 87, 129, 130, 140, 149, 170, 196, 209, 210, 252, 254, 263, 264, 266, 278, 281, 288, 297, 321, 327, 336, 369, 418, 461, 470, 476, 492, 503, 542, 561, 566, 572, 606, 616, 626, 627, 633], 127: [26, 77, 96, 126, 151, 178, 198, 215, 269, 279, 302, 357, 387, 416, 471, 515, 532, 534, 562, 583, 586, 609], 167: [20, 22, 25, 64, 72, 75, 86, 141, 144, 155, 166, 190, 193, 243, 282, 313, 316, 319, 348, 356, 381, 390, 393, 398, 431, 454, 466, 468, 469, 478, 485, 489, 513, 514, 548, 569, 580, 595, 615, 621], 523: [7, 152, 176, 235, 253, 276, 332, 338, 367, 401, 456, 490, 497, 508, 564, 588, 630]}

# imports 
import random

# function to generate a random path order (consider as genome) for each depot # returns a dict 
def randomPathArrangement(cluster):
    randomPathArr = {}
    customerAllocation = cluster
    for depot in depot_locations:
        # append depot as key and list of randomly arranged customer nodes to randomPathArr
        randomPathArr.update({depot:random.sample(customerAllocation[depot], len(customerAllocation[depot]))})
    return randomPathArr

# function to generate multiple initial solutions (population) # returns genomes (individual solution) in a list
def population_init(cluster, size):
    population = []
    for i in range(0, size):
        population.append(randomPathArrangement(cluster))
    return population

# return populationFitness at each index...
def fitness(population, edges):

    # to store each weight per traversal made between two nodes (when calculating traversal distance per depot)
    genomeWeight = []

    # for a possible solution in the set of solutions (genome in population)
    for genome in population:
        # initialise genomeFitness dict to store distances at each depot per genome 
        genomeFitness = {}
        for depot in genome.keys():
        #for depot in depot_locations:
            genomeFitness.update({depot:[]})
            nodes = genome[depot]
            # loop for length of nodes
            for idx in range(len(nodes)-1):
                # set nodeA to current loop index
                nodeA = nodes[idx]
                # set nodeB to next loop index
                nodeB = nodes[idx+1]
                # get sum of each traversal distance occurence between two nodes | routing() enables this 
                distance = sum(routing(nodeA, nodeB, edges)[2])
                genomeFitness[depot].append(distance)
        # update popFitness list with each genome and relative fitness values
        genomeWeight.append(genomeFitness)

    genomeWeights = []
    for genome in genomeWeight:
        # initialise popWeight list to store total weight at each depot for genome
        depotWeight = []
        # loop through each depot (chromosome per genome)
        for depot in genome:
            depotWeight.append({depot:sum(genome[depot])})
        genomeWeights.append(depotWeight)
    
    # to store the weight per index (weight per genome) [each solution indicated by index i.e., 0, 1, 2 etc.]
    weightIndex = {}
    for idx, genome in enumerate(genomeWeights):
        tempIndexStore = []
        for depot in genome:
            for i in depot.values():
                tempIndexStore.append(i)
        weightIndex.update({idx:sum(tempIndexStore)})
    
    return population, weightIndex

# selection() uses weighted selection probability for returning 2 cluster arrangement (2 genomes)
def selection(population, popWeights):
    # selectionPair defines a list containing # of genomes selected through concept of roulette-wheel (in this case k=2). 
    selectionPair = random.choices(population=population, weights=popWeights, k=2)
    # return the selectionPair (in form list)
    return selectionPair 

# crossover() takes two routes (genomes) and performs crossover operation to produce new routes (offspring)
def crossover(selectionPair):
    # initialise routeA (i.e., genome 1)
    routeA = selectionPair[0]
    # initialise routeB (i.e., genome 2)
    routeB = selectionPair[1]
    
    # check if both routes (both genomes) have same number of keys (depots)
    if len(routeA.keys()) != len(routeB.keys()):
        # give error warning
        print(f'#---crossover error---# route depots are not of same length')
    # if route keys are of same length then proceed with crossover
    else:
        # set random crossover point nPoint
        nPoint = random.randint(1, len(routeA.keys())-1)
        # get routeA.values() up to nPoint 
        extractRouteA1 = list(routeA.values())[nPoint:]
        # get routeB.values() up to nPoint
        extractRouteB1 = list(routeB.values())[nPoint:]
        # get routeA.values() beyond nPoint
        extractRouteA2 = list(routeA.values())[:nPoint]
        # get routeB.values() beyond nPoint
        extractRouteB2 = list(routeB.values())[:nPoint]

        # set offsprings as corresponding concantenation of each route extracts
        offspringA = extractRouteB2+extractRouteA1
        offspringB = extractRouteA2+extractRouteB1

        # intialise offspring dicts
        newRouteA = {}
        newRouteB = {}

        # loop through each depot nodes
        for i, depot in enumerate(depot_locations):
            # allocate new routes to relative depots
            newRouteA.update({depot:offspringA[i]})
            newRouteB.update({depot:offspringB[i]})

    # return the new offsprings
    return newRouteA, newRouteB 

# mutation() selects a random depot in a route and re-arranges the order of nodes at this depot 
def mutation(crossedPair, mutationRate):
    # get genome routeA
    routeA = crossedPair[0]
    # get genome routeB
    routeB = crossedPair[1]
    # get MUTATION_RATE
    MUTATION_RATE = mutationRate
    # randomly generate a decisive float
    decision = random.uniform(0, 1)
    
    # check if mutation is active
    if decision < MUTATION_RATE:
        # select random mutation point
        mutationPoint = random.choice(depot_locations)
        # extract routes of the mutation point (extract nodes from depot)
        toMutateRouteA = routeA[mutationPoint]
        toMutateRouteB = routeB[mutationPoint]
        # mutate routes (re-arrange the node order) 
        mutateRouteA = random.sample(toMutateRouteA, k=len(toMutateRouteA))
        mutateRouteB = random.sample(toMutateRouteB, k=len(toMutateRouteB))
        # set the mutated routes as offsprings 
        routeA[mutationPoint] = mutateRouteA
        routeB[mutationPoint] = mutateRouteB
        # return mutated routes
        return routeA, routeB

    # other wise return non-mutated routes
    return routeA, routeB

# helper function to elect two routes (with poor fitness/largest ovr. distance) to be replaced with the new off spring
def replacePopulation(population, popWeight, newPair):

    # sort the population index in descending order via weight (index of longest distance first)
    sortPopWeight = sorted(popWeight.items(), key=lambda idx: idx[1], reverse=True)

    # initialise popIdxToReplace list to store indexes of the less fit genomes to be replaced 
    popIdxToReplace = []
    for sortedWeightIdx in sortPopWeight:
        # check for the number of genomes to be replaced 
        if len(popIdxToReplace) != int(len(newPair)):
            popIdxToReplace.append(sortedWeightIdx[0])
    
    # verify replacement constraints
    if len(popIdxToReplace)==len(newPair):
        # enumerate popIdxToReplace, so that each population index can be replaced with the corresponding index of newPair
        for i, popIdx in enumerate(popIdxToReplace):
            # replace population index with corresponding index of the offspring routes
            population[popIdx] = newPair[i]
    
    # return the updated population
    return population

# helper function to retain fitter population of routes in the next iterations (preserves 50% from previous iteration)
def retainBestRoutes(currentPopulation, popWeight):

    # sort the population index in ascending order (shortest ovr. distance to largest over. distance)
    sortPopWeight = sorted(popWeight.items(), key=lambda idx: idx[1])
    
    # initilaise popRetainIdx list to store indexes of population to retain
    popIdxToRetain = []
    for sortedWeightIdx in sortPopWeight:
        # check loop for 50% of the popluation 
        if len(popIdxToRetain) != int(len(currentPopulation)/2):
            popIdxToRetain.append(sortedWeightIdx[0])
    
    # initialise popRetain list to store list of population
    popRetain = []
    # match popIdx with currentPopulation to append to popRetain list
    for popIdx in popIdxToRetain:
        popRetain.append(currentPopulation[popIdx])
    
    # return list of population to retain for next iteration
    return popRetain 

# helper function to elicit best route per iteration 
def getBestRoute(currentPopulation, popWeight):
    # sort weights in ascending order and extract the index of the shortest overall route
    sortPopWeight = sorted(popWeight.items(), key=lambda idx: idx[1])
    bestRouteIdx, routeDistance = sortPopWeight[0][0], sortPopWeight[0][1]
    bestRoute = currentPopulation[bestRouteIdx]
    return bestRoute, routeDistance

# run genetic algorithm to search for the most optimal route; termination criteria = iterations [keep 'size' EVEN int]
def runGeneticAlgorithm(cluster, size, iterationLimit, edges):
    
    # generate initial population 
    population = population_init(cluster, size)
    # initialise list to store upcoming population
    nextPopulation = []
    # set iteration count to 0
    iteration = 0

    # while iteration limit not exceeded
    while iteration < iterationLimit:
        # print(f'\n\niteration: {iteration}')
        # check for retained population 
        if nextPopulation:
            population = nextPopulation
            # if population list has enough set of routes 
            if int(len(population)) != int(len(population)*2):
                # generate half the number of initial size to append to the existing list of population NB! key[0] to unpack list
                population.append(population_init(cluster, int(size/2))[0])
            # print(f'population: {population}')
        
        # get initial fitness of each routes and assign the value as weights per genome 
        popWeights = fitness(population, edges)[1]
        # select two parent genomes (two routes)
        selectionPair = selection(population, popWeights)
        # perform the crossover operation 
        crossedPair = crossover(selectionPair)
        # mutation stage and not 'mutated' because mutation occurs at random rates
        mutationStagePair = mutation(crossedPair, mutationRate=0.4)
        # replace the population with new offsprings at the corresponding indexes
        population = replacePopulation(population, popWeights, mutationStagePair)
        # get the new fitness per route in the population index
        popWeights = fitness(population, edges)[1]
        # filter populations to preserve '50%' of the fittest routes in the current population for next iteration
        nextPopulation = retainBestRoutes(population, popWeights)
        print(f'nextPopulation: {nextPopulation}')

        # iteration increment
        iteration += 1
    
    # reset popWeights after loop to eval. new pop
    popWeights = fitness(population, edges)[1]

    # gets the shortest route found and the route distance
    bestRoute = getBestRoute(population, popWeights)

    # return the bestRoute along with route distance (tuple)
    return bestRoute


# dummy prints: for example purpose
# store the results of runGeneticAlgorithm() in getRoute (iteration set to low number due to time constraints for developer's hardware specs.)
getRoute = runGeneticAlgorithm(cluster=dummyCluster, size=4, iterationLimit=2, edges=edges)
# get best route found by the G.A.
bestRoute = getRoute[0]
# get the distance of the best route
routeDistance = getRoute[1]
# display route and distance
print(f'bestRoute: {bestRoute}\n\ndistance: {routeDistance}')

7. FINDINGS AND COMPARISON OF GREEDY SEARCH APPROACH AND GENETIC ALGORITHM 

# Performance:
(NB: the data below were collected from testing during code development)
(NB: time to complete execution ('time taken') was retrieved from in-cell time tabs in jupyter notebook cells)
(NB: since each task had different constriants, only comparable variables have been elected for this performance comparison)

# Greedy Search
- Distance converged: 20374.176350449245 
- Path cost: 194640.17
- Iterations taken: N/A (doesn't apply in this context)
- Time taken: 9.4s

# Genetic Algorithm 
- Distance converged: 47063.76523902698
- Path cost: 633431.05 
- Iterations taken: 2
- Time taken: 3m 52.2s 

In [ ]:
# relevant imports 

import matplotlib.pyplot as plt
import numpy as np
import math 

gsDistance = 20374.176350449245
gsPathCost = 194640.17

gaDistance = 47063.76523902698
gaPathCost = 633431.05 

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])

algorithms = ['Greedy Search', 'Genetic Algorithm']
distance = [gsDistance, gaDistance]
ax.bar(algorithms, distance)
plt.show()

The results for each tasks are available on task1_solution.json and task2_solution.json respectively. (May need to execute the actual code files first for these outputs to be made)

- References: 

(1) https://peerj.com/articles/cs-55.pdf 
(2) https://data-flair.training/blogs/advantages-of-python-pandas/ 
(3) https://numpy.org/devdocs/user/absolute_beginners.html#:~:text=NumPy%20arrays%20are%20faster%20and,to%20be%20optimized%20even%20further. 
(4) https://www.javatpoint.com/pandas-vs-numpy 
(5) https://medium.com/fintechexplained/why-should-we-use-numpy-c14a4fb03ee9#:~:text=NumPy%20is%20an%20open%2Dsource,%2C%20statistical%2C%20and%20algebraic%20routines. 
(6) https://networkx.org/ 
(7) https://docs.scipy.org/doc/scipy/reference/spatial.distance.html 
(8) http://sbubeck.com/bubeck09a.pdf 
(9) https://www.geeksforgeeks.org/building-an-undirected-graph-and-finding-shortest-path-using-dictionaries-in-python/ 
(10) https://www.techopedia.com/definition/16931/greedy-algorithm 
[11] D. M. Chickering, Optimal Strucure Identification With Greedy Search, journal of machine learning research 3 (2002). [online] pp. 507-544
[12] https://link.springer.com/chapter/10.1007/0-387-28356-0_4  
[13] https://www.sciencedirect.com/topics/computer-science/wheel-selection#:~:text=The%20roulette%20wheel%20selection%20method,total%20fitness)%20of%20each%20individual. 
[14] https://mathworld.wolfram.com/LocalMinimum.html#:~:text=A%20local%20minimum%2C%20also%20called,may%20be)%20a%20global%20minimum. 